## Import the modules

In [1]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.cross_validation import train_test_split
from sklearn.feature_selection import RFE
%matplotlib inline

C:\Users\JAMESWhitfield\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Functions for loading the data and building the different classes of model

In [2]:
def import_WDS_results(filepath):
    
    # Read in data
    df = pd.read_csv(filepath)
    
    # Read in data
    df_original = pd.read_csv(filepath)
    
    # Fill in NaN with the mean
    #df.loc[:, df.columns != 'Skill Level'] = df.loc[:, df.columns != 'Skill Level'].fillna(df.mean())
    
    # Fill in NaN with the mean
    df.loc[:, ['Activity Data Score',
               'Feedback Data Score',
              'BadgeData Data Score',
              'Checklist Data Score',
              'Competency Beginning Data Score',
              'Competency Progressing Data Score',
              'Competency Proficient Data Score',
              'Competency Mastery Data Score',
              'AboutMe Data Score',
              'CVs Data Score']] = df.loc[:, ['Activity Data Score',
               'Feedback Data Score',
              'BadgeData Data Score',
              'Checklist Data Score',
              'Competency Beginning Data Score',
              'Competency Progressing Data Score',
              'Competency Proficient Data Score',
              'Competency Mastery Data Score',
              'AboutMe Data Score',
              'CVs Data Score']].fillna(df.mean())
    
    # Fill in NaN with the 0
    df.loc[:, ['Achievement Data Score',
               'Courses Data Score',
               'Engagement Minimum Data Score',
              'Engagement Low Data Score',
              'Engagement Medium Data Score',
              'Engagement High Data Score',
              'Learning Course Data Score',
              'KCs Data Score']] = df.loc[:, ['Achievement Data Score',
               'Courses Data Score',
               'Engagement Minimum Data Score',
              'Engagement Low Data Score',
              'Engagement Medium Data Score',
              'Engagement High Data Score',
              'Learning Course Data Score',
              'KCs Data Score']].fillna(0)
    
    # Generate total competency score
    if 'Competency Beginning Data Score' in df.columns:
        df['Competency Data Score'] = df['Competency Beginning Data Score'].fillna(0) + \
        (df['Competency Progressing Data Score'].fillna(0)*2) + \
        (df['Competency Proficient Data Score'].fillna(0)*4) + \
        (df['Competency Mastery Data Score'].fillna(0)*8)

        # Fill in NaN where no competencies are provided
        df.loc[pd.isna(df['Competency Beginning Data Score']) & 
                pd.isna(df['Competency Progressing Data Score']) &
                pd.isna(df['Competency Proficient Data Score']) &
                pd.isna(df['Competency Mastery Data Score']), 'Competency Data Score'] = None
    else:
        df = df
        
    # Generate total engagement score
    if 'Engagement Minimum Data Score' in df.columns:
        df['Engagement Data Score'] = df['Engagement Minimum Data Score'].fillna(0) + \
        (df['Engagement Low Data Score'].fillna(0)*2) + \
        (df['Engagement Medium Data Score'].fillna(0)*4) + \
        (df['Engagement High Data Score'].fillna(0)*8)

        # Fill in NaN where no engagement scores are provided
        df.loc[pd.isna(df['Engagement Minimum Data Score']) & 
                pd.isna(df['Engagement Low Data Score']) &
                pd.isna(df['Engagement Medium Data Score']) &
                pd.isna(df['Engagement High Data Score']), 'Engagement Data Score'] = None
    else:
        df = df
    
    # Reset the index
    df = df.reset_index()
    
    # Reset the index
    df_original = df_original.reset_index()
    
    return df, df_original

In [3]:
def naive_bayes_model(df, feature_cols):
    
    # Drop the NaN in the training data
    df = df.dropna(subset=['Skill Level']).copy()
    
    # Split dataset in features and target variable
    X = df[feature_cols] # Features
    y = df['Skill Level'] # Target variable
    
    # Split dataset into training and test data
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=0)
    
    # Instantiate the model (using the default parameters)
    gnb = GaussianNB()
    
    # Define feature selection
    #selector = RFE(gnb, step=1)

    # Fit the model with data
    gnb.fit(X_train,y_train)

    # Make predictions
    y_pred = gnb.predict(X_test)
    y_pred = pd.DataFrame(y_pred)
    y_pred = y_pred.rename(index=str, columns={0: "Predictions"})
    
    # Confidence
    confidence = gnb.predict_proba(X_test)
    confidence = confidence.max(axis=1)
    confidence_df = pd.DataFrame(confidence)
    confidence_df = confidence_df.rename(index=str, columns={0: "Confidence"})
    y_pred = pd.merge(y_pred, confidence_df, left_index=True, right_index=True)
    
    # Convert y datasets to pandas dataframes
    y_train = pd.DataFrame(data=y_train)
    y_test = pd.DataFrame(data=y_test)
    
    # Set index to match y_test
    index_vals = y_test.index.values
    y_pred = y_pred.set_index(index_vals)
    
    #selections = selector.support_
    
    return X_train, X_test, y_train, y_test, y_pred, gnb

In [4]:
def import_keyword_results(filepath,sheet):
    
    # Read in data
    df = pd.read_excel(io=filepath, sheet_name=sheet)
    
    # Drop irrelevant columns
    df = df.drop(columns=['Activity Data Score', 'Achievement Data Score','Courses Data Score','Engagement Data Score',
                'Feedback Data Score','BadgeData Data Score','Competency Beginning Data Score', 
                'Competency Progressing Data Score','Competency Proficient Data Score','Competency Mastery Data Score',
                'Learning Course Data Score','AboutMe Data Score','CVs Data Score'])
    
    return df

In [5]:
skill_mapping = {
    'Purchase Price Allocation (PPA)': (1, 'Purchase Price Allocation (PPA)'),
    'Valuation': (2, 'Valuation'),
    'Impairment Testing': (3, 'Impairment Testing'),
    'Equity Incentives - Tax Valuati': (4, 'Equity Incentives - Tax Valuation'),
    'Cost Benefit Analysis': (5, 'Cost Benefit Analysis'),
    'Due Diligence': (6, 'Due Diligence'),
    'Real estate sell-side preparati': (7, 'Real estate sell-side preparation'),
    'Modelling': (8, 'Modelling'),
    'Debt Options Analysis': (9, 'Debt Options Analysis'),
    'Bid financial evaluation': (10, 'Bid financial evaluation'),
    'Green Book business cases': (11, 'Green Book business cases'),
    'Power Purchase Agreements (PPA': (12, 'Power Purchase Agreements (PPAs)'),
    'Working Capital & Liquidity Imp': (13, 'Working Capital & Liquidity Improvement'),
    'Financial Analysis Due Diligenc': (14, 'Financial Analysis Due Diligence'),
    'Advanced Excel': (15, 'Advanced Excel'),
    'Cash flow Forecasting': (16, 'Cash flow Forecasting'),
    'Formal Insolvency': (17, 'Formal Insolvency'),
    'Equity to Enterprise Value Brid': (18, 'Equity to Enterprise Value Bridges'),
    'EBITDA Adjustment Identificatio': (19, 'EBITDA Adjustment Identification'),
    'Debt and debt-like items analys': (20, 'Debt and debt-like items analysis'),
    'Stock Exchange Working Capital': (21, 'Stock Exchange Working Capital'),
    'Working capital target /peg/ normalised analysis': (22, 'Working capital target peg norm'),
    'Project Management': (23, 'Project Management')
    }

In [6]:
def calculate_results(filepath_WDS,skill,feature_cols,filepath_keyword,skill_mapping):
    
    # Read in data
    df, df_original = import_WDS_results(filepath_WDS)
    df = df.drop(['index'], axis=1)
    df_original = df_original.drop(['index'], axis=1)
    
    # Build the model
    X_train, X_test, y_train, y_test, y_pred, gnb = naive_bayes_model(df, feature_cols)

    # Create table of test predictions
    predictions_test = pd.merge(df, y_pred, left_index=True, right_index=True)
    predictions_test['Test Flag'] = 1
    predictions_test
    
    # Create table of predictions
    X = df[feature_cols]
    y_pred = pd.DataFrame(gnb.predict(X))
    y_pred = y_pred.rename(index=str, columns={0: "Predictions"})
    
    # Confidence
    confidence = gnb.predict_proba(X)
    max_confidence = confidence.max(axis=1)
    max_confidence_df = pd.DataFrame(max_confidence)
    max_confidence_df = max_confidence_df.rename(index=str, columns={0: "Confidence"})
    y_pred = pd.merge(y_pred, max_confidence_df, left_index=True, right_index=True)
    
    # Reset Index of y_pred
    index_vals = df.index.values
    y_pred = y_pred.set_index(index_vals)
    
    # Other probabilities
    #confidence_df = pd.DataFrame(confidence, columns=['1','2','3','4','5'])
    #y_pred = pd.merge(y_pred, confidence_df, left_index=True, right_index=True)
    
    # Merge test flag with predictions
    predictions = pd.merge(df_original, y_pred, left_index=True, right_index=True)
    if 'CVs Data Score' in predictions.columns:
        predictions = predictions.sort_values(by=['CVs Data Score'],ascending=False)
    else:
        predictions = predictions.sort_values(by=['WDS Score'],ascending=False)
    m=predictions_test[['Custom ID','Test Flag']]
    predictions = predictions.merge(m , left_on='Custom ID', right_on='Custom ID', how='left')
    
    # Create Secondary Prediction
    #twomax = predictions.loc[:,['1', '2', '3', '4', '5']]
    #twomax['Secondary Prediction'] = (twomax.rank(axis=1, ascending=False) == 2).idxmax(axis=1).astype(int)
    #twomax['Secondary Confidence'] = twomax[twomax.rank(axis=1, ascending=False) == 3].max(axis=1)
    #twomax = twomax.loc[:,['Secondary Prediction','Secondary Confidence']]
    #predictions = pd.merge(predictions, twomax, left_index=True, right_index=True)
    #predictions = predictions.rename(index=str, columns={"1": "Probability of 1", 
                                                         #"2": "Probability of 2", 
                                                         #"3": "Probability of 3",
                                                         #"4": "Probability of 4",
                                                         #"5": "Probability of 5"})
    
    # Load keywords matched
    df_keyword = import_keyword_results(filepath_keyword,skill)
    
    # Merge keywords with the predictions
    final_df = predictions.merge(df_keyword , left_on='Custom ID', right_on='Custom ID', how='left')
    final_df['Skill_ID'] = skill_mapping.get(skill)[0]
    final_df['Skill_Name'] = skill_mapping.get(skill)[1]
    
    # Generate Result Column
    final_df['Result_Calc'] = (final_df['Skill Level'] - final_df['Predictions'])**2
    final_df['Result'] = 'Wrong'
    for index, row in final_df.iterrows():
        result = ''
        if row['Result_Calc'] < 0.5:
            result = 'Correct'
        elif row['Result_Calc'] < 2:
            result = 'Within 1'
        elif row['Result_Calc'] < 5:
            result = 'Within 2'
        elif row['Result_Calc'] < 10:
            result = 'Within 3'
        elif row['Result_Calc'] < 17:
            result = 'Within 4'
        else:
            result = ''
        final_df.set_value(index,'Result',result)
    final_df.drop(columns=['Result_Calc'],inplace=True)

    return final_df, selections, y_pred

## Train and run the models

In [7]:
# Inputs
skill_list = ['Due Diligence', 'Project Management', 'Real estate sell-side preparati', 'Formal Insolvency', 'Debt Options Analysis', 'Modelling','Impairment Testing','Advanced Excel','Purchase Price Allocation (PPA)','Working Capital & Liquidity Imp','Cash flow Forecasting','Financial Analysis Due Diligenc']
feature_cols = ['Activity Data Score','Achievement Data Score','Courses Data Score','Engagement Data Score','Feedback Data Score','Checklist Data Score','Learning Course Data Score','AboutMe Data Score','CVs Data Score','KCs Data Score']
filepath_keyword = 'C:/Users/JAMESWhitfield/Documents/01 - Projects/15 - Ernst & Young/Models/Training_Data/PeopleFeatures_311018_13FeaturesWithKeywordsTitleBoostPurgedConceptsCV_AllSkills.xls'

final_df = pd.DataFrame()
#selections = pd.DataFrame(columns = ['Skill'] + feature_cols)

for skill_i in skill_list:
    
    filepath_WDS = 'C:/Users/JAMESWhitfield/Documents/01 - Projects/15 - Ernst & Young/Models/Training_Data/151118/PeopleFeatures_151118_' + skill_i + '.csv'
    skill = skill_i
    
    current_df, current_selections, y_pred = calculate_results(filepath_WDS,skill,feature_cols,filepath_keyword,skill_mapping)
    
    print(skill_i + " completed...")
    
    #print(current_selections)
    
    final_df = final_df.append(current_df)
    
    #new_row = {
        #"Skill": skill_i,
        #"Activity Data Score": current_selections[0],
        #"Achievement Data Score": current_selections[1],
        #"Courses Data Score": current_selections[2],
        #"Engagement Data Score": current_selections[3],
        #"Feedback Data Score": current_selections[4],
        #"Checklist Data Score": current_selections[5],
        #"Learning Course Data Score": current_selections[6],
        #"AboutMe Data Score": current_selections[7],
        #"CVs Data Score": current_selections[8],
        #"KCs Data Score": current_selections[9]
    #}
    
    #selections = selections.append(new_row,ignore_index=True)

C:\Users\JAMESWhitfield\Anaconda3\lib\site-packages\ipykernel_launcher.py:82: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


Due Diligence completed...
Empty DataFrame
Columns: [Skill, Activity Data Score, Achievement Data Score, Courses Data Score, Engagement Data Score, Feedback Data Score, Checklist Data Score, Learning Course Data Score, AboutMe Data Score, CVs Data Score, KCs Data Score]
Index: []
Project Management completed...
Empty DataFrame
Columns: [Skill, Activity Data Score, Achievement Data Score, Courses Data Score, Engagement Data Score, Feedback Data Score, Checklist Data Score, Learning Course Data Score, AboutMe Data Score, CVs Data Score, KCs Data Score]
Index: []
Real estate sell-side preparati completed...
Empty DataFrame
Columns: [Skill, Activity Data Score, Achievement Data Score, Courses Data Score, Engagement Data Score, Feedback Data Score, Checklist Data Score, Learning Course Data Score, AboutMe Data Score, CVs Data Score, KCs Data Score]
Index: []
Formal Insolvency completed...
Empty DataFrame
Columns: [Skill, Activity Data Score, Achievement Data Score, Courses Data Score, Engag

In [8]:
# Write results to a csv
final_df.to_csv('15112018_Naive_Bayes_Results_v1.csv', sep=',')

In [9]:
selections

,Skill,Activity Data Score,Achievement Data Score,Courses Data Score,Engagement Data Score,Feedback Data Score,Checklist Data Score,Learning Course Data Score,AboutMe Data Score,CVs Data Score,KCs Data Score
